### import

In [1]:
%load_ext autoreload

In [15]:
%aimport utils.search_address

In [51]:
import pandas as pd 
import numpy as np
import math
import re

from icecream import ic
from tqdm.notebook import tqdm 

from utils.custom_utils import load_var, save_var
from utils.webscraping import ArticleScraper
from utils.search_address import CheckAddressLevels
import utils.search_address as sa

from importlib import reload

df_addr = pd.read_parquet('./address/df_addr.parquet.gzip')
df_text = pd.read_parquet('./dataframe/flasher_hk_20130101_20220307.gzip')


In [17]:
%autoreload 2

### CAL: CheckAddressLevels

In [5]:
index = 133
text_iter = iter(zip(df_text.index[index:], df_text.loc[index:, 'content']))

In [6]:
index, text = next(text_iter)
text = sa.clean_text(text)

In [7]:
self = CheckAddressLevels(df_address=df_addr, text=text)
result = self.process()
print(result)

충남 사리


In [8]:
df_brief = self.df_brief.copy()
df_brief

,original_name,found_name,level,count,upper
2,사리,사리,4,1,충남
1,서산시,서산시,2,1,충남
0,충남,충남,0,1,
3,대전,대전,0,1,


In [14]:
text_mark = text
for item in df_brief['found_name'].values:
    ic(item)
    text_mark = sa.re_sub(item, text_mark)

print(f"\nindex: {index}\n")
print(text_mark)

ic| item: '사리'
ic| item: '서산시'
ic| item: '충남'
ic| item: '대전'



index: 133

엄태용 / 사진 = 한화이글스 제공
10대 여고생에게 수면제를 먹인 뒤 성폭행한 혐의로 구속 기소된 전 프로야구 한화이글스 선수 엄태용(25)이 항소심 후 형량이 오히려 늘어났다.🔺대전🔻고법 제1형사부(부장 이준명)는 14일 아동·청소년의 성보호에 관한 법률위반 혐의로 기소된 엄태용의 항소심에서 징역 3년6개월의 원심을 파기, 징역 4년6개월을 선고했다.여기에 더해 40시간의 성폭력 방지 강의 수강과 아동·청소년 관련기관 등에 5년간의 취업 제한을 명령했다.엄씨는 지난해 6월 SNS로 피해자인 여고생 A씨를 알게 된 후 🔺충남🔻 🔺서산시🔻 자신의 자택에서 성폭행한 혐의로 기소돼 1심에서 징역 3년6개월을 선고 받았다. 1심 재판부의 판결에 검찰은 "양형이 가벼워 부당" 엄씨는 "형량이 무겁다"고 각각 항소했으나 재판부는 검찰의 항소 이유만 받아줬다. 재판부는 "성적 해소를 위해 🔺사리🔻분별 능력이 부족한 청소년을 계획적으로 수면제를 먹이고 항거불능 상태에서 성폭행한 죄질이 불량하다"고 판시했다. 이어 "잘못을 반성하지 않고 변명으로 일관하며 피해자와 그 가족의 정신적 고통이 큰 상태지만 범죄처벌 전력이 없는 점 등을 종합적으로 고려했다"고 양형 이유를 밝혔다. 한편 엄태용은 지난 2016년 9월에도 자신의 여자친구 집에서 "말을 듣지 않았다"란 이유로 막대기로 여자친구의 엉덩이를 수차례 때려 벌금 300만원 형에 선고를 받기도 했다.장지민 한경닷컴 기자 newsinfo@hankyung.com


#### extract sentences having the found keywords

In [ ]:
text_mark = text
for item in df_brief['found_name'].values:
    ic(item)
    text_mark_result = sa.extract_sentences_with_keyword(keyword=item, text=text_mark)
    text_mark = ','

print(f"\nindex: {index}\n")
print(text_mark)


In [12]:
sentences = re.split(r'(?<=[.!?])+', text_mark)
# keyword_pattern = re.escape(f'🔺*🔻')
# pattern = r'\((.*?)\)'
keyword_pattern = r'🔺(.*?)🔻'
# matches = re.findall(pattern, text)
keyword_sentences = [sentence.strip() for sentence in sentences if re.search(keyword_pattern, sentence, re.IGNORECASE)]
keyword_sentences

['🔺대전🔻고법 제1형사부(부장 이준명)는 14일 아동·청소년의 성보호에 관한 법률위반 혐의로 기소된 엄태용의 항소심에서 징역 3년6개월의 원심을 파기, 징역 4년6개월을 선고했다.',
 '엄씨는 지난해 6월 SNS로 피해자인 여고생 A씨를 알게 된 후 🔺충남🔻 🔺서산시🔻 자신의 자택에서 성폭행한 혐의로 기소돼 1심에서 징역 3년6개월을 선고 받았다.',
 '재판부는 "성적 해소를 위해 🔺사리🔻분별 능력이 부족한 청소년을 계획적으로 수면제를 먹이고 항거불능 상태에서 성폭행한 죄질이 불량하다"고 판시했다.']

In [20]:
text_marked = sa.extract_sentences_marked(marked_text=text_mark)
print(text_marked)

🔺대전🔻고법 제1형사부(부장 이준명)는 14일 아동·청소년의 성보호에 관한 법률위반 혐의로 기소된 엄태용의 항소심에서 징역 3년6개월의 원심을 파기, 징역 4년6개월을 선고했다. 엄씨는 지난해 6월 SNS로 피해자인 여고생 A씨를 알게 된 후 🔺충남🔻 🔺서산시🔻 자신의 자택에서 성폭행한 혐의로 기소돼 1심에서 징역 3년6개월을 선고 받았다. 재판부는 "성적 해소를 위해 🔺사리🔻분별 능력이 부족한 청소년을 계획적으로 수면제를 먹이고 항거불능 상태에서 성폭행한 죄질이 불량하다"고 판시했다.


### CAL: all dataframe

In [27]:
df_text.head(1)

,범죄 유형,지역,위도,경도,중복 여부,퀄리티,제외 여부,기사제목,사건 장소,수사 기관,본문,URL,일자,언론사,기고자,비고,selenium_html,content,found_sentences
0,NaN,인천 부평구,NaN,NaN,NaN,NaN,NaN,"""같이 살자"" 2억 뜯어낸 채팅녀 알고 보니 20대 남자였다",2억여원,"이대,검찰,한경닷컴 객원,서울서부지법,재판부,법원,형사항",채팅앱에서 만난 남성들에게 자신을 여성이라고 속인 뒤 2억여원을 가로챈 20대 남성...,https://www.hankyung.com/society/article/20220...,20220207,한국경제,이보배,NaN,"<body>\n\t<div id=""wrap"" class=""view"">\n\n\t\t...",\n\n\n\n\nA씨는 2020년 초부터 수 개월간 채팅앱에서 만난 남성들에게 자...,


In [52]:
df_text['지역'] = df_text['지역'].astype(str)

In [54]:
df_text['found_sentences'] = ''

In [80]:
df_text['found_text'] = ''

In [81]:
df_text.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377 entries, 0 to 376
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   범죄 유형            0 non-null      float64
 1   지역               376 non-null    object 
 2   위도               0 non-null      float64
 3   경도               0 non-null      float64
 4   중복 여부            0 non-null      float64
 5   퀄리티              0 non-null      float64
 6   제외 여부            0 non-null      float64
 7   기사제목             377 non-null    object 
 8   사건 장소            328 non-null    object 
 9   수사 기관            377 non-null    object 
 10  본문               377 non-null    object 
 11  URL              209 non-null    object 
 12  일자               377 non-null    int64  
 13  언론사              377 non-null    object 
 14  기고자              286 non-null    object 
 15  비고               0 non-null      float64
 16  selenium_html    209 non-null    object 
 17  content         

In [91]:
start_index = 10
end_index = len(df_text)
total_length = len(df_text.iloc[start_index:end_index])
for index, row in tqdm(df_text[start_index:end_index].iterrows(), total=total_length):
    text = row['content']
    if isinstance(text, str) and len(text) > 0:
        text = sa.clean_text(text)

        self = CheckAddressLevels(df_address=df_addr, text=text)
        result = self.process()
        df_text.at[index, '지역'] = result
        
        text_mark = text
        for item in self.df_brief['found_name'].values:
            text_mark = sa.re_sub(item, text_mark)
        df_text.at[index, 'found_text'] = text_mark
        
        text_marked = sa.extract_sentences_marked(marked_text=text_mark)
        df_text.at[index, 'found_sentences'] = text_marked

  0%|          | 0/367 [00:00<?, ?it/s]

#### inspection

In [65]:
df_text.iloc[:1]

,범죄 유형,지역,위도,경도,중복 여부,퀄리티,제외 여부,기사제목,사건 장소,수사 기관,본문,URL,일자,언론사,기고자,비고,selenium_html,content,found_sentences
0,NaN,서울,NaN,NaN,NaN,NaN,NaN,"""같이 살자"" 2억 뜯어낸 채팅녀 알고 보니 20대 남자였다",2억여원,"이대,검찰,한경닷컴 객원,서울서부지법,재판부,법원,형사항",채팅앱에서 만난 남성들에게 자신을 여성이라고 속인 뒤 2억여원을 가로챈 20대 남성...,https://www.hankyung.com/society/article/20220...,20220207,한국경제,이보배,NaN,"<body>\n\t<div id=""wrap"" class=""view"">\n\n\t\t...",\n\n\n\n\nA씨는 2020년 초부터 수 개월간 채팅앱에서 만난 남성들에게 자...,7일 법조계에 따르면 🔺서울🔻서부지법 형사항소2부(부상준 부장판사)는 사기·공갈·절...


In [66]:
df_text.loc[0, 'found_sentences']

'7일 법조계에 따르면 🔺서울🔻서부지법 형사항소2부(부상준 부장판사)는 사기·공갈·절도·여신전문금융업법 위반·컴퓨터등사용사기 등 혐의로 재판에 넘겨진 A씨(24)에게 1심과 같은 징역 5년을 선고했다.'

In [83]:
filter_columns = ['지역', 'found_sentences', 'found_text']

In [92]:
df_text[df_text['found_sentences'] != ''][filter_columns]

,지역,found_sentences,found_text
0,서울,7일 법조계에 따르면 🔺서울🔻서부지법 형사항소2부(부상준 부장판사)는 사기·공갈·절...,A씨는 2020년 초부터 수 개월간 채팅앱에서 만난 남성들에게 자신을 23살 여성이...
1,서울 서울특별시,지난달 초 🔺서울🔻 모 여고에서 군인에게 위문 편지를 보내는 과정에서 군인 비하 문...,지난달 초 🔺서울🔻 모 여고에서 군인에게 위문 편지를 보내는 과정에서 군인 비하 문...
2,대전,🔺대전🔻지법 형사항소5부(이경희 부장판사)는 강제추행 혐의로 기소된 A씨(33)에 ...,🔺대전🔻지법 형사항소5부(이경희 부장판사)는 강제추행 혐의로 기소된 A씨(33)에 ...
4,대구 북구,🔺대구🔻지방법원 포항지원 제1형사부(권순향 재판장)는 아동청소년의성보호에관한법률위반...,가출 여고생에게 숙식을 제공하며 성매매를 강요한 일당이 실형을 선고받았다. 이들 중...
5,경기 의정부시,19일 법조계에 따르면 🔺의정부🔻지법 형사합의11부는 아동·청소년의 성 보호에 관한...,피해자의 진술 번복과 사건 직후 CCTV에 담긴 이해하기 힘든 행동으로 인해 강간 ...
...,...,...,...
267,인천 부평구,🔺인천🔻부평경찰서는 폭력행위 등 처벌에 관한 법률 위반 혐의로 여고생 A(18) 양...,부평 묻지마 폭행 / 부평 묻지마 폭행 YTN 방송 캡처\n부평 묻지마 폭행부평 묻...
268,인천 부평구,"24일 🔺인천🔻 부평경찰서는 ""폭력행위 등 처벌에 관한 법률 위반 혐의로 여고생 A...",부평 묻지마 폭행 / 부평 묻지마 폭행 YTN 방송 캡처\n'부평 묻지마 폭행' 사...
269,인천 부평구,24일 🔺인천🔻 부평경찰서에 따르면 전날 오후 ‘부평 묻지마 폭행’ 피의자인 최모 ...,"부평 묻지마 폭행 가해자, 신상 강제 공개…네티즌 분노 유발한 글 보니 \n \n부..."
270,인천 부평구,"24일 🔺인천🔻 부평경찰서는 ""폭력행위 등 처벌에 관한 법률 위반 혐의로 여고생 A...",부평 묻지마 폭행 / 부평 묻지마 폭행 YTN 방송 캡처\n부평 묻지마 폭행'부평 ...


In [85]:
df_text.iloc[:5][filter_columns]

,지역,found_sentences,found_text
0,서울,7일 법조계에 따르면 🔺서울🔻서부지법 형사항소2부(부상준 부장판사)는 사기·공갈·절...,
1,서울 서울특별시,지난달 초 🔺서울🔻 모 여고에서 군인에게 위문 편지를 보내는 과정에서 군인 비하 문...,
2,대전,,대전지법 형사항소5부(이경희 부장판사)는 강제추행 혐의로 기소된 A씨(33)에 대해...
3,None,,/사진=유튜브 승무원 룩북 영상 캡처\n'승무원 룩북' 영상으로 논란이 불거진 유튜...
4,대구 북구,,가출 여고생에게 숙식을 제공하며 성매매를 강요한 일당이 실형을 선고받았다. 이들 중...


In [86]:
df_text.iloc[2]['found_text']

'대전지법 형사항소5부(이경희 부장판사)는 강제추행 혐의로 기소된 A씨(33)에 대해 무죄로 판단한 원심을 파기하고 벌금 500만원을 선고했다. 사진은 기사와 무관함. /사진=게티이미지뱅크 \n10대 여학생 뒤에서 몰래 소변을 본 30대 남성이 1·2심에서 추행 혐의 무죄 판결을 받았다가 파기환송심에서 벌금형을 선고받았다. 대전지법 형사항소5부(이경희 부장판사)는 강제추행 혐의로 기소된 A씨(33)에 대해 무죄로 판단한 원심을 파기하고 벌금 500만원을 선고했다고 26일 밝혔다. 재판부는 40시간의 성폭력 치료 프로그램 이수와 3년간 아동 관련 기관 취업 제한도 함께 명령했다. A씨는 2019년 11월25일 오후 11시께 충남 지역 한 아파트 놀이터 의자에 앉아 휴대전화로 통화를 하던 피해자 B씨(당시 18세)의 머리카락, 후드티, 패딩점퍼 위에 몰래 소변을 본 혐의로 기소됐다. 당시에는 피해 사실을 몰랐던 B씨는 집에 돌아간 뒤에야 머리카락과 옷에 소변이 묻어있는 것을 발견해 경찰에 신고했다. 앞서 1·2심 재판부는 A씨에 대해 "피고인의 행위로 인해 피해자의 성적 자기결정의 자유가 침해됐다고 인정하기 부족하다"면서 무죄를 선고했다. 하지만 지난해 11월 대법원은 "A씨의 행위는 객관적으로 일반인에게 성적 수치심이나 혐오감을 일으키게 하고 선량한 도덕관념에 반하는 행위"라고 판단했다. 또 "피해자의 성적 자기결정권을 침해하는 추행 행위에 해당한다고 볼 여지가 있다"며 사건을 대전지법으로 돌려보냈다. 결국 파기환송심 재판부는 대법원의 판단에 따라 사건을 다시 심리했고, "피해자가 추행을 당하는 사실을 명확히 인지하지 못했더라도 강제추행죄는 성립한다"는 취지로 벌금형을 선고했다. 이보배 한경닷컴 객원기자 newsinfo@hankyung.com'

In [102]:
type(df_text.loc[:, '지역'].iloc[3])

NoneType

#### inspection - ii

In [59]:
index = 0
text_iter = iter(zip(df_text.index[index:], df_text.loc[index:, 'content']))

In [60]:
index, text = next(text_iter)
text = sa.clean_text(text)

In [61]:
self = CheckAddressLevels(df_address=df_addr, text=text)
result = self.process()
print(result)

서울


In [62]:
df_brief = self.df_brief.copy()
df_brief

,original_name,found_name,level,count,upper
1,서울,서울,0,1,


In [63]:
text_mark = text
for item in df_brief['found_name'].values:
    ic(item)
    text_mark = sa.re_sub(item, text_mark)

print(f"\nindex: {index}\n")
print(text_mark)

ic| item: '서울'



index: 0

A씨는 2020년 초부터 수 개월간 채팅앱에서 만난 남성들에게 자신을 23살 여성이라고 소개한 뒤 함께 살자고 접근해 생활비 등의 명목으로 돈을 뜯어냈다. 사진은 기사와 무관함. /사진=게티이미지뱅크 
채팅앱에서 만난 남성들에게 자신을 여성이라고 속인 뒤 2억여원을 가로챈 20대 남성이 항소심에서도 실형을 선고받았다. 7일 법조계에 따르면 🔺서울🔻서부지법 형사항소2부(부상준 부장판사)는 사기·공갈·절도·여신전문금융업법 위반·컴퓨터등사용사기 등 혐의로 재판에 넘겨진 A씨(24)에게 1심과 같은 징역 5년을 선고했다. A씨는 2020년 초부터 수 개월간 채팅앱에서 만난 남성들에게 자신을 23살 여성이라고 소개했다. 그는 남성들에게 교제를 하자거나 함께 살자고 접근해 생활비 등의 명목으로 돈을 뜯어낸 혐의로 재판에 넘겨졌다. A씨는 피해자 B씨에게 "같이 살 집을 구하자. 보증금, 살림살이를 구입하는데 필요한 돈은 내가 관리하겠다"고 속여 2주 만에 3260만원을 편취했다. 또 다른 피해자 C씨에게는 "사귀자"고 접근해 돈을 빌리고, 계좌번호와 비밀번호 등 개인정보를 건네 받아 은행에서 대신 대출받는 방식으로 총 1730만원을 빼앗았다. 피해자 D씨에게는 음란행위 영상을 받은 뒤 "네가 일하는 곳에 영상을 뿌리겠다"고 협박해 410만원을 갈취하고, 수백만원을 대출받아 빼돌리기도 했다. 1심 재판부는 "범행 수법과 방법 등이 상당히 불량하고, 사회적 비난의 여지가 큰 점 등을 불리한 정상으로 참작한다"면서 징역 5년을 선고했다. 이후 A씨는 형이 너무 무겁다며, 검창른 너무 가볍다며 각각 항소했지만 항소심 재판부의 판단도 다르지 않았다. 항소심 재판부는 "피해자가 수십 명이고, 피해액 합계가 2억4000만원에 이르는데도 피해 회복을 위해 별다른 노력을 하지 않았다"고 지적했다. 이어 "절도 등으로 여러 차례 소년보호처분을 받은 전력이 있고, 재판 중에도 다른 미결수용자를 폭행하는 등 규율위반 행위로 금치 30일 처분을 받기도 했다"면서 "다만 잘못

#### save to parquet

In [103]:
df_text.to_parquet('./dataframe/flasher_hk_20130101_20220307_cal.parquet.gzip', compression='gzip')

### Coodinates

In [16]:
import pandas as pd 
import numpy as np
import math
import re

from icecream import ic
from tqdm.notebook import tqdm 

from utils.custom_utils import load_var, save_var
from utils.webscraping import ArticleScraper
from utils.search_address import CheckAddressLevels
import utils.search_address as sa

from importlib import reload

df_addr = pd.read_parquet('./address/df_addr.parquet.gzip')
df_text = pd.read_parquet('./dataframe/flasher_hk_20130101_20220307_cal.parquet.gzip')


#### ask: filter non empty strings

In [2]:
import pandas as pd

def filter_non_empty_strings(df: pd.DataFrame, select_column: str) -> pd.DataFrame:
    """
    Filters the DataFrame to include only rows where the values in the specified
    column are non-empty strings.

    Parameters:
    df (pd.DataFrame): The input DataFrame.
    select_column (str): The column to filter on.

    Returns:
    pd.DataFrame: A filtered DataFrame with only non-empty string values in the specified column.
    """
    return df[df[select_column].apply(lambda x: isinstance(x, str) and x != '' and x != 'nan')]

In [118]:
# Example usage
df_text = pd.DataFrame({
    'column1': ['a', '', 'b', None, 'c'],
    'column2': [1, 2, 3, 4, 5]
})

select_column = 'column1'
filtered_df = filter_non_empty_strings(df_text, select_column)
print(filtered_df)

  column1  column2
0       a        1
2       b        3
4       c        5


In [3]:
select_column = '지역'
filtered_df = filter_non_empty_strings(df_text, select_column)

In [7]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 104 entries, 0 to 271
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   범죄 유형            0 non-null      float64
 1   지역               104 non-null    object 
 2   위도               0 non-null      float64
 3   경도               0 non-null      float64
 4   중복 여부            0 non-null      float64
 5   퀄리티              0 non-null      float64
 6   제외 여부            0 non-null      float64
 7   기사제목             104 non-null    object 
 8   사건 장소            92 non-null     object 
 9   수사 기관            104 non-null    object 
 10  본문               104 non-null    object 
 11  URL              104 non-null    object 
 12  일자               104 non-null    int64  
 13  언론사              104 non-null    object 
 14  기고자              100 non-null    object 
 15  비고               0 non-null      float64
 16  selenium_html    104 non-null    object 
 17  content          104 

#### get_lat_long()

In [8]:
loc_name = filtered_df.loc[0, select_column]
lat, long = sa.get_lat_long(loc_name)

In [9]:
print(lat, long)

37.5666791 126.9782914


In [12]:
filtered_df.index

Index([  0,   1,   2,   4,   5,   7,   8,   9,  10,  11,
       ...
       262, 263, 264, 265, 266, 267, 268, 269, 270, 271],
      dtype='int64', length=104)

In [13]:
total_length = len(filtered_df)
for index, row in tqdm(filtered_df.iterrows(), total=total_length):
    lat, long = sa.get_lat_long(row['지역'])
    filtered_df.at[index, '위도'] = lat
    filtered_df.at[index, '경도'] = long

  0%|          | 0/104 [00:00<?, ?it/s]

Location '충북 지리' not found.


In [17]:
update_column = ['위도', '경도']
df_text.loc[:, update_column] = filtered_df[update_column]

In [18]:
df_text.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377 entries, 0 to 376
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   범죄 유형            0 non-null      float64
 1   지역               334 non-null    object 
 2   위도               103 non-null    float64
 3   경도               103 non-null    float64
 4   중복 여부            0 non-null      float64
 5   퀄리티              0 non-null      float64
 6   제외 여부            0 non-null      float64
 7   기사제목             377 non-null    object 
 8   사건 장소            328 non-null    object 
 9   수사 기관            377 non-null    object 
 10  본문               377 non-null    object 
 11  URL              209 non-null    object 
 12  일자               377 non-null    int64  
 13  언론사              377 non-null    object 
 14  기고자              286 non-null    object 
 15  비고               0 non-null      float64
 16  selenium_html    209 non-null    object 
 17  content         

In [21]:
df_text.head()

,범죄 유형,지역,위도,경도,중복 여부,퀄리티,제외 여부,기사제목,사건 장소,수사 기관,본문,URL,일자,언론사,기고자,비고,selenium_html,content,found_sentences,found_text
0,NaN,서울,37.566679,126.978291,NaN,NaN,NaN,"""같이 살자"" 2억 뜯어낸 채팅녀 알고 보니 20대 남자였다",2억여원,"이대,검찰,한경닷컴 객원,서울서부지법,재판부,법원,형사항",채팅앱에서 만난 남성들에게 자신을 여성이라고 속인 뒤 2억여원을 가로챈 20대 남성...,https://www.hankyung.com/society/article/20220...,20220207,한국경제,이보배,NaN,"<body>\n\t<div id=""wrap"" class=""view"">\n\n\t\t...",\n\n\n\n\nA씨는 2020년 초부터 수 개월간 채팅앱에서 만난 남성들에게 자...,7일 법조계에 따르면 🔺서울🔻서부지법 형사항소2부(부상준 부장판사)는 사기·공갈·절...,A씨는 2020년 초부터 수 개월간 채팅앱에서 만난 남성들에게 자신을 23살 여성이...
1,NaN,서울 서울특별시,37.554555,126.970779,NaN,NaN,NaN,'군인비하' 여고생 위문편지도 봉사활동 인정될까?,"서울,서울강서양천교육지원청,여고","국민신문고,서울시교육청,사나,국군,특,서울강서양천교육지원청,별시,교육청",지난달 초 서울 모 여고에서 군인에게 위문 편지를 보내는 과정에서 군인 비하 문구를...,https://www.hankyung.com/society/article/20220...,20220204,한국경제,이미나,NaN,"<body>\n\t<div id=""wrap"" class=""view"">\n\n\t\t...",\n\n\n\n\n지난달 초 서울 모 여고에서 군인에게 위문 편지를 보내는 과정에서...,지난달 초 🔺서울🔻 모 여고에서 군인에게 위문 편지를 보내는 과정에서 군인 비하 문...,지난달 초 🔺서울🔻 모 여고에서 군인에게 위문 편지를 보내는 과정에서 군인 비하 문...
2,NaN,대전,36.349701,127.384902,NaN,NaN,NaN,여고생 등에 '몰래 소변' 30대男 무죄→벌금형 뒤집혔다,충남,"한경닷컴 객원,형사항소,당하,재판부,대법원,대전지법",10대 여학생 뒤에서 몰래 소변을 본 30대 남성이 1 2심에서 추행 혐의 무죄 판...,https://www.hankyung.com/society/article/20220...,20220126,한국경제,이보배,NaN,"<body>\n\t<div id=""wrap"" class=""view"">\n\n\t\t...",\n\n\n\n\n대전지법 형사항소5부(이경희 부장판사)는 강제추행 혐의로 기소된 ...,🔺대전🔻지법 형사항소5부(이경희 부장판사)는 강제추행 혐의로 기소된 A씨(33)에 ...,🔺대전🔻지법 형사항소5부(이경희 부장판사)는 강제추행 혐의로 기소된 A씨(33)에 ...
3,NaN,None,NaN,NaN,NaN,NaN,NaN,"""'승무원 룩북' 유튜버, 입던 속옷 돈 받고 팔려고 "" 폭로","적나라,아프리카,룩북","미국,유튜브,청구,정보통신,유튜버,대한항공,VVIP",'승무원 룩북' 영상으로 논란이 불거진 유튜버가 자신이 입던 속옷을 나눠준다는 이벤...,https://www.hankyung.com/life/article/20211222...,20211222,한국경제,김소연,NaN,"<body>\n\t<div id=""wrap"" class=""view"">\n\n\t\t...",\n\n\n\n\n/사진=유튜브 승무원 룩북 영상 캡처\n'승무원 룩북' 영상으로 ...,,/사진=유튜브 승무원 룩북 영상 캡처\n'승무원 룩북' 영상으로 논란이 불거진 유튜...
4,NaN,대구 북구,35.876282,128.597118,NaN,NaN,NaN,가출 여고생 '성매매' 강요 20대男 공범 여친 몰래 '성폭행' 하기도,"아동청소년의성보호에관한법률,경기,인천,북구,포항지원","애플,한경닷컴 객원,대구지방법원,재판부,제1형사부,포항시",가출 여고생에게 숙식을 제공하며 성매매를 강요한 일당이 실형을 선고받았다. 이들 중...,https://www.hankyung.com/society/article/20211...,20211219,한국경제,이보배,NaN,"<body>\n\t<div id=""wrap"" class=""view"">\n\n\t\t...",\n\n\n\n\n가출 여고생에게 숙식을 제공하며 성매매를 강요한 일당이 실형을 선...,🔺대구🔻지방법원 포항지원 제1형사부(권순향 재판장)는 아동청소년의성보호에관한법률위반...,가출 여고생에게 숙식을 제공하며 성매매를 강요한 일당이 실형을 선고받았다. 이들 중...


#### save to parquet

In [22]:
df_text.to_parquet('./dataframe/flasher_hk_20130101_20220307_loc.parquet.gzip', compression='gzip')

In [24]:
df_text.to_excel('./dataframe/flasher_hk_20130101_20220307_loc.xlsx', index=False)

# end